RQ1: Is there an association between repository activity characteristics (number of commits and number of contributors) and GUI end-to-end testing intensity (number of GUI test files and number of GUI tests) in non-trivial repositories?

### Imports

In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from scipy import stats
from statsmodels.stats.multitest import multipletests

### Load CSV files

In [108]:
repository = pd.read_csv("E2EGit/repository.csv")
non_trivial = pd.read_csv("E2EGit/non_trivial_repository.csv")
gui_repo = pd.read_csv("E2EGit/gui_testing_repo_details.csv")
gui_tests = pd.read_csv("E2EGit/gui_testing_test_details.csv")
perf_tests = pd.read_csv("E2EGit/performance_testing_test_details.csv")

repository.columns = repository.columns.str.strip()
non_trivial.columns = non_trivial.columns.str.strip()
perf_tests.columns = perf_tests.columns.str.strip()

repository = repository.rename(columns={"name": "repository_name",})
non_trivial = non_trivial.rename(columns={"name": "repository_name",})

#### Prepare analysis dataframe

In [109]:
gui_repo_non_trivial = gui_repo.merge(
    non_trivial[["repository_name"]],
    on="repository_name",
    how="inner"
)

gui_by_repo = gui_repo_non_trivial[["repository_name", "number_of_files", "number_of_tests"]].rename(
    columns={"number_of_files": "gui_test_files", "number_of_tests": "gui_tests"}
)

df_analysis = gui_by_repo.merge(repository, on="repository_name", how="inner")

# only repos with at least one GUI test file and at least one GUI test
df_analysis = df_analysis[(df_analysis["gui_test_files"] > 0) & (df_analysis["gui_tests"] > 0)]

print("Repositories:", df_analysis["repository_name"].nunique())
df_analysis.head()


In [124]:
df_analysis = gui_by_repo.merge(repository, on="repository_name", how="inner")
df_analysis = df_analysis[(df_analysis["gui_test_files"] > 0) & (df_analysis["gui_tests"] > 0)]
df_analysis.head()

### Descriptive Statistics

In [ ]:
columns = ["commits", "contributors", "gui_test_files", "gui_tests"]

desc_main = df_analysis[columns].agg(["count", "mean", "median", "std", "min", "max"])
display(desc_main)

print("Min commits/contributors:")
display(df_analysis[["commits", "contributors"]].min())

print("Top 5 repositories by GUI tests:")
display(df_analysis.nlargest(5, "gui_tests")[["commits", "contributors", "gui_test_files", "gui_tests"]])

### Normality Tests

Shapiro-Wilk

In [126]:
columns = ["commits", "contributors", "gui_test_files", "gui_tests"]
normality_results = {}

print("RQ1:")
for column in columns:
    x = df_analysis[column].dropna()
    stat, p_value = stats.shapiro(x)
    is_normal = p_value > 0.05
    normality_results[column] = is_normal

    print(f"  {column:15s}: W={stat:.4f}, p={p_value:.4f} | "
          f"{'Normal' if is_normal else 'Not Normal'}")

D’Agostino–Pearson

In [121]:
for column in columns:
    x = df_analysis[column].dropna()
    n = len(x)

    if n < 8:
        print(f"  {column:15s}: N={n:4d} | N/A (requires N >= 8)")
        continue

    stat, p_value = stats.normaltest(x)
    is_normal = p_value > 0.05

    print(f"  {column:15s}: N={n:4d}, K2={stat:.4f}, p={p_value:.4g} | "
          f"{'Normal' if is_normal else 'Not Normal'}")

### Spearman correlations

In [122]:
# commits vs gui_test_files
tmp = df_analysis[["commits", "gui_test_files"]].dropna()
rho_commits_files, p_commits_files = spearmanr(tmp["commits"], tmp["gui_test_files"])

# commits vs gui_tests
tmp = df_analysis[["commits", "gui_tests"]].dropna()
rho_commits_tests, p_commits_tests = spearmanr(tmp["commits"], tmp["gui_tests"])

# contributors vs gui_test_files
tmp = df_analysis[["contributors", "gui_test_files"]].dropna()
rho_contrib_files, p_contrib_files = spearmanr(tmp["contributors"], tmp["gui_test_files"])

# contributors vs gui_tests
tmp = df_analysis[["contributors", "gui_tests"]].dropna()
rho_contrib_tests, p_contrib_tests = spearmanr(tmp["contributors"], tmp["gui_tests"])

rho_commits_files, p_commits_files, rho_commits_tests, p_commits_tests, rho_contrib_files, p_contrib_files, rho_contrib_tests, p_contrib_tests


### Results table

In [127]:
results = pd.DataFrame([
    {"activity": "commits", "intensity": "gui_test_files", "rho": rho_commits_files, "p": p_commits_files},
    {"activity": "commits", "intensity": "gui_tests",      "rho": rho_commits_tests, "p": p_commits_tests},
    {"activity": "contributors", "intensity": "gui_test_files", "rho": rho_contrib_files, "p": p_contrib_files},
    {"activity": "contributors", "intensity": "gui_tests",      "rho": rho_contrib_tests, "p": p_contrib_tests},
])

results["p_holm"] = multipletests(results["p"], method="holm")[1]

results


### Hexbin plot

In [128]:
def make_two_panel_hexbin(df, outpath="rq1_hexbin_two_panel.png", gridsize=35):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    fig.subplots_adjust(right=0.88, wspace=0.25)

    pairs = [
        ("commits", "gui_test_files", "log10(commits)", "log10(GUI test files)", "Commits vs GUI test files"),
        ("commits", "gui_tests",      "log10(commits)", "log10(GUI tests)",      "Commits vs GUI tests"),
    ]

    hbs = []
    for ax, (xcolumn, ycolumn, xl, yl, title) in zip(axes, pairs):
        x = df[xcolumn].to_numpy()
        y = df[ycolumn].to_numpy()
        lx = np.log10(x + 1)
        ly = np.log10(y + 1)


        hb = ax.hexbin(lx, ly, gridsize=gridsize, bins='log', mincnt=1)
        hbs.append(hb)
        ax.set_xlabel(xl)
        ax.set_ylabel(yl)
        ax.set_title(title)

    cax = fig.add_axes([0.90, 0.15, 0.02, 0.70]) 
    cbar = fig.colorbar(hbs[0], cax=cax)
    cbar.set_label("Bin count (log scale)")

    fig.savefig(outpath, dpi=300, bbox_inches="tight")
    plt.show()

make_two_panel_hexbin(df_analysis)